In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=b68a977a0678b5ac6aa5d959646ae0da9f918ce498b821f48bbfb55ca0d693bb
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import collect_list, array_distinct, explode, split, col

# Step 1: Create SparkSession
spark = SparkSession.builder.appName("FPGrowthExample").getOrCreate()

# Step 2: Read data (assuming CSV format for this example)
data = spark.read.csv("groceries_data.csv", header=True, inferSchema=True)

# Step 3: Group data based on member number
grouped_data = data.groupBy("Member_number").agg(collect_list("itemDescription").alias("Items"))

# Step 4: Show grouped data
grouped_data.show(truncate=False)

# Step 5: Add a column 'basket' with unique items
grouped_data = grouped_data.withColumn("basket", array_distinct(grouped_data["Items"]))

# Step 6: Show data again
grouped_data.show(truncate=False)

# Step 7: Explode the Items array to separate items into rows
exploded_data = grouped_data.select("Member_number", explode("Items").alias("item"))

# Step 8: Replace '/' with ',' in the items to separate

separated_data = exploded_data.withColumn("item", explode(split("item", "/")))

# Step 9: Group the separated items back into lists and ensure they are unique
final_data = separated_data.groupBy("Member_number").agg(collect_list("item").alias("Items"))

# Step 10: Ensure Items are unique again
final_data = final_data.withColumn("Items", array_distinct(col("Items")))

# Step 11: Show the final separated data
final_data.show(truncate=False)

# Step 12: Create FPGrowth model with specified parameters
minSupport = 0.1
minConfidence = 0.2

fp = FPGrowth(minSupport=minSupport, minConfidence=minConfidence, itemsCol='Items', predictionCol='prediction')

# Step 13: Fit FPGrowth model to the final data
model = fp.fit(final_data)

# Step 14: Show frequent itemsets
model.freqItemsets.show(10)  # Show top 10 frequent itemsets

# Step 15: Filter association rules based on confidence
filtered_rules = model.associationRules.filter(model.associationRules.confidence > 0.4)

# Step 16: Show filtered rules
filtered_rules.show(truncate=False)

# Step 17: Create a new DataFrame for predictions
new_data = spark.createDataFrame(
    [
        (["vegetable juice", "frozen fruits", "packaged fruit"],),
        (["mayonnaise", "butter", "buns"],)
    ],
    ["Items"]  # Changed to "Items"
)

# Step 18: Show new data for predictions
new_data.show(truncate=False)

# Step 19: Transform the model with the new data for predictions
predictions = model.transform(new_data)

# Step 20: Show predictions
predictions.show(truncate=False)

spark.stop()

+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Member_number|Items                                                                                                                                                                                                                                                                                                          |
+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1000         |[soda, canned beer, sausa

In [9]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!pip install pyspark==3.0.0
!pip install graphframes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 20.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044158 sha256=bde5d814fbc5a8665beb38c5ca38b1e4174a64f38ac0abde3190b57b42a5d8a5
  Stored in directory: /root/.cache/pip/wheels/b1/bb/8b/ca24d3f756f2ed967225b0871898869db676eb5846df5adc56
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.3
    Uninstalling pyspark-3.5.3:
      Successfully uninstalled pyspark-3.5.3


In [1]:


# Step 2: Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

# Step 3: Create SparkSession
from pyspark.sql import SparkSession
from graphframes import GraphFrame
from pyspark.sql.functions import desc

# Create SparkSession with GraphFrames
spark = SparkSession.builder \
    .appName("Graph Analytics") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12") \
    .getOrCreate()

# Step 4: Create Vertices DataFrame
vertices = spark.createDataFrame([
    ("Alice", 45),
    ("Jacob", 43),
    ("Roy", 21),
    ("Ryan", 49),
    ("Emily", 24),
    ("Sheldon", 52)
], ["id", "age"])

# Step 5: Create Edges DataFrame
edges = spark.createDataFrame([
    ("Sheldon", "Alice", "Sister"),
    ("Alice", "Jacob", "Husband"),
    ("Emily", "Jacob", "Father"),
    ("Ryan", "Alice", "Friend"),
    ("Alice", "Emily", "Daughter"),
    ("Jacob", "Roy", "Son"),
    ("Roy", "Ryan", "Son")
], ["src", "dst", "relation"])

# Step 6: Create a GraphFrame
graph = GraphFrame(vertices, edges)

# Step 7: Graph Analytics
print("Grouped and ordered edges:")
graph.edges.groupBy("src", "dst").count().orderBy(desc("count")).show()

print("Filtered edges where src or dst is 'Alice':")
graph.edges.where("src = 'Alice' OR dst = 'Alice'").groupBy("src", "dst").count().orderBy(desc("count")).show()

print("Subgraph where src or dst is 'Alice':")
subgraph_edges = graph.edges.where("src = 'Alice' OR dst = 'Alice'")
subgraph = GraphFrame(graph.vertices, subgraph_edges)
subgraph.edges.show()

print("Finding motifs in the graph:")
motifs = graph.find("(a) - [ab] -> (b)")
motifs.show()

print("Calculating PageRank:")
rank = graph.pageRank(resetProbability=0.15, maxIter=5)
rank.vertices.orderBy(desc("pagerank")).show()

print("In-Degree of nodes:")
in_degree = graph.inDegrees
in_degree.orderBy(desc("inDegree")).show()

print("Out-Degree of nodes:")
out_degree = graph.outDegrees
out_degree.orderBy(desc("outDegree")).show()

print("Finding connected components:")
spark.sparkContext.setCheckpointDir("/tmp/checkpoints")  # Required for connected components
cc = graph.connectedComponents()
cc.show()

print("Finding strongly connected components:")
scc = graph.stronglyConnectedComponents(maxIter=5)
scc.show()

print("Performing BFS from node 'Alice' to node 'Roy':")
bfs_result = graph.bfs(fromExpr="id = 'Alice'", toExpr="id = 'Roy'", maxPathLength=2)
bfs_result.show()

# Stop the Spark session
spark.stop()


Grouped and ordered edges:
+-------+-----+-----+
|    src|  dst|count|
+-------+-----+-----+
|  Alice|Jacob|    1|
|Sheldon|Alice|    1|
|  Emily|Jacob|    1|
|  Alice|Emily|    1|
|  Jacob|  Roy|    1|
|   Ryan|Alice|    1|
|    Roy| Ryan|    1|
+-------+-----+-----+

Filtered edges where src or dst is 'Alice':
+-------+-----+-----+
|    src|  dst|count|
+-------+-----+-----+
|  Alice|Jacob|    1|
|Sheldon|Alice|    1|
|  Alice|Emily|    1|
|   Ryan|Alice|    1|
+-------+-----+-----+

Subgraph where src or dst is 'Alice':
+-------+-----+--------+
|    src|  dst|relation|
+-------+-----+--------+
|Sheldon|Alice|  Sister|
|  Alice|Jacob| Husband|
|   Ryan|Alice|  Friend|
|  Alice|Emily|Daughter|
+-------+-----+--------+

Finding motifs in the graph:
+-------------+--------------------+-----------+
|            a|                  ab|          b|
+-------------+--------------------+-----------+
|  {Jacob, 43}|   {Jacob, Roy, Son}|  {Roy, 21}|
|  {Emily, 24}|{Emily, Jacob, Fa...|{Jacob, 4